In [ ]:
import glob
import pandas as pd

In [ ]:
CSV_DIR = "C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\\raw\csv\\03-11"
COLS = [
    "Flow ID",
    " Source IP",
    " Source Port",
    " Destination IP",
    " Destination Port",
    " Protocol",
    " Timestamp",
    " Flow Duration",
    " Label",
]

In [ ]:
def read_csv(dir: str, cols: list = None):
    print("READING CSV FILES...")

    extension = "*.csv"
    files = glob.glob(f"{dir}/{extension}")

    dfs = []
    for file in files:
        print(f"READING {file}")
        if cols:
            dfs.append(pd.read_csv(file, usecols=cols))
        else:
            dfs.append(pd.read_csv(file))

    print("MERGING CSV FILES...")
    df = pd.concat(dfs, ignore_index=True)
    df.columns = df.columns.str.strip()

    print('DROPPING ROWS WITH MISSING "Flow ID"...')
    df = df.drop(df[pd.isnull(df["Flow ID"])].index)

    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    min_timestamp = df["Timestamp"].min()

    return df, min_timestamp

In [ ]:
df, _ = read_csv(CSV_DIR, COLS)
df

In [ ]:
df = df.rename(columns={"Timestamp": "stime", "Flow Duration": "dur"})

In [ ]:
df["stime"] = df["stime"].apply(lambda x: x.timestamp())

In [ ]:
df.head(5)

In [ ]:
output_file = "C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\clean\labeled_sample.csv"
counter = 0
for chunk in pd.read_csv("C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\clean\sample.csv", chunksize=100_000):
    temp1 = pd.merge(chunk, df, how="left", left_on=["src_ip", "src_p", "dst_ip", "dst_p", "protocol.1",], right_on=["Source IP", "Source Port", "Destination IP", "Destination Port", "Protocol"])
    temp2 = pd.merge(chunk, df, how="left", left_on=["src_ip", "src_p", "dst_ip", "dst_p", "protocol.1",], right_on=["Destination IP", "Destination Port", "Source IP", "Source Port", "Protocol"])
    combine = pd.concat([temp1, temp2])
    combine.drop_duplicates(inplace=True)
    combine = combine[
        (combine["stime"] <= combine["timestamp"])
        & (combine["timestamp"] <= combine["stime"] + combine["dur"])
    ]
    combine = combine.drop(columns=["stime", "dur", "timestamp", "Flow ID", "src_ip", "src_p", "dst_ip", "dst_p", "protocol.1", "Source IP", "Source Port", "Destination IP", "Destination Port", "Protocol"])
    combine = combine.rename(columns={"Label": "label"})
    if counter == 0:
        combine.to_csv(output_file, index=False, mode='w', header=True)
    else:
        combine.to_csv(output_file, index=False, mode='a', header=False)
    counter += 1
    if counter == 100:
        break

In [ ]:
df_check = pd.read_csv("C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\clean\labeled_sample.csv")

In [ ]:
df_check.label.unique()

In [ ]:
df_check.label.value_counts()